In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import tqdm
import seaborn as sns
sns.set_theme()

In [2]:
import os
#add .. to path via os
os.chdir('..')
print(os.getcwd())

/home/airlab/Documents/NLP-Project


In [3]:
from project.models.lit_modules import SeqAnnotator
from project.dataset.ChessDataset import ChessDataset

In [4]:
WEIGHT_LOCATION = "./checkpoints/lstm-best-v3.ckpt"
WEIGHT_LOCATION = "/home/airlab/Documents/NLP-Project/checkpoints/result_seqs/lstm-best-v3.ckpt"

# Load the model
model = SeqAnnotator.load_from_checkpoint(WEIGHT_LOCATION)
model.eval()

SeqAnnotator(
  (embedding): Embedding(1968, 100)
  (rnn): LSTM(100, 512, batch_first=True, bidirectional=True)
  (fc_out): Sequential(
    (0): Linear(in_features=1024, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=3, bias=True)
    (3): Softmax(dim=-1)
  )
  (loss_fn): CrossEntropyLoss()
  (reg_loss_fn): MSELoss()
)

In [5]:
#Load the dataset
dataset = ChessDataset(
    parquette_path="./data/games_0001/train_100K.parquet",
    inputColumns=["Moves"],
    labelColumns=[]
)

Loading parquet file @  ./data/games_0001/train_100K.parquet  with columns  ['Moves']
Loaded 70000 rows and 1 columns


Converting columns to indices: 100%|██████████| 1/1 [00:00<00:00,  1.51it/s]


In [6]:
data = [dataset[i][0][0] for i in range(len(dataset))]

In [7]:
tensors = []
for d in tqdm.tqdm(data):
    # Convert the data to a tensor
    d = torch.tensor(d).unsqueeze(0)
    tensors.append(d)


  0%|          | 0/70000 [00:00<?, ?it/s]/tmp/ipykernel_547541/983101555.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  d = torch.tensor(d).unsqueeze(0)
100%|██████████| 70000/70000 [00:00<00:00, 123529.74it/s]


In [8]:
torch.unsqueeze(dataset[0][0][0], dim=0).shape

torch.Size([1, 163])

In [10]:
outcome, (embeddings_h, embeddings_c) = model.forward(torch.unsqueeze(dataset[0][0][0], dim=0).cuda())
embeddings_c.shape

torch.Size([2, 1, 512])